<a href="https://colab.research.google.com/github/PedroHAndrade/Tradutor-de-Idiomas/blob/main/Tradutor_de_Idiomas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q google-genai
!pip install -q gradio
!pip install -q SpeechRecognition pydub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 11.7 MB/s eta 0:00:00


In [5]:
import os
from google.colab import userdata
from google import genai
import gradio as gr
import speech_recognition as sr
import io
import numpy as np

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

client = genai.Client()

modelo_traducao = "gemini-2.0-flash"

def traduzir_texto(texto, idioma_destino):
    """Função para traduzir o texto usando o Gemini."""
    prompt = f"Traduza o seguinte texto, de forma sucinta, e com alguns outros exemplos que poderiam ser usados para a mesma frase para {idioma_destino}: '{texto}'"
    resposta = client.models.generate_content(model=modelo_traducao, contents=prompt)
    return resposta.text

def transcrever_audio(audio_data):
    """Função para transcrever o áudio em texto."""
    if audio_data is None:
        return "Nenhum áudio gravado."
    rate, data = audio_data
    recognizer = sr.Recognizer()
    try:
        # Converter os dados de áudio para o formato que o AudioData espera
        audio = sr.AudioData(data.tobytes(), rate, 2) # Taxa e largura como argumentos posicionais
        texto_transcrito = recognizer.recognize_google(audio, language="pt-BR") # Reconhece em português
        return texto_transcrito
    except sr.UnknownValueError:
        return "Não foi possível entender o áudio."
    except sr.RequestError as e:
        return f"Erro ao solicitar resultado do serviço de reconhecimento de fala; {e}"

def traduzir_audio(audio_data, idioma_destino):
    """Função para transcrever e depois traduzir o áudio."""
    texto_transcrito = transcrever_audio(audio_data)
    print(f"Texto transcrito (para debug): {texto_transcrito}")
    if "Não foi possível entender o áudio" in texto_transcrito or "Erro ao solicitar resultado" in texto_transcrito or "Nenhum áudio gravado" in texto_transcrito:
        return texto_transcrito
    return traduzir_texto(texto_transcrito, idioma_destino)

# INTERFACE
iface_texto = gr.Interface(
    fn=traduzir_texto,
    inputs=[
        gr.Textbox(label="Texto a ser traduzido"),
        gr.Dropdown(choices=["português", "inglês", "espanhol", "francês", "italiano", "turco", "mandarim", "aramaico"], label="Idioma de destino")
    ],
    outputs=gr.Textbox(label="Tradução (do Texto)"),
    title="Tradução de Texto"
)

iface_audio = gr.Interface(
    fn=traduzir_audio,
    inputs=[
        gr.Audio(sources=["microphone"], label="Gravar Áudio"),
        gr.Dropdown(choices=["português", "inglês", "espanhol", "francês", "italiano", "turco", "mandarim", "aramaico"], label="Idioma de destino")
    ],
    outputs=gr.Textbox(label="Tradução (do Áudio)"),
    title="Tradução de Áudio"
)

# Cria a interface com abas
demo = gr.TabbedInterface([iface_texto, iface_audio], ["Texto", "Áudio"])

# Inicia a interface
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2db03b373eac29624f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Texto transcrito (para debug): Boa tarde me chamo Pedro
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2db03b373eac29624f.gradio.live
